In [40]:
import math
def calWelding(element, workStations, jointLength=12.2, weldingSpeed= 300/1000*24*60, weldingWireOD=2/1000, bevelAngle=(20*3.1459)/180, numberOfWeldMachines=2):
    wallThickness = element["wallThickness"]
    od = element["od"]
    weldVolumeSpeed = math.pow(weldingWireOD,2)*math.pi/4*weldingSpeed
    capHeight = 2/46.8 * wallThickness
    capArea = math.tan(bevelAngle)*wallThickness*capHeight
    weldArea = wallThickness*math.tan(bevelAngle)*wallThickness
    weldVolumeNeeded = (capArea+weldArea)*od*math.pi
    weldRate = weldVolumeNeeded/weldVolumeSpeed/numberOfWeldMachines #time of welding one pipe
    numberOfWeldingStations = len([i for i in workStations if i['Name'] == 'Welding'])
    durationPerStation = weldRate / numberOfWeldingStations
    print(f"Welding duration is {weldRate*24*60} min")
    print(f"Welding per station is {durationPerStation*24*60} min")
    for ws in workStations:
        if ws["Name"] == "Welding":
            ws["Duration"] = durationPerStation*24*60
    cycleTime = max([i["Duration"] for i in workStations])  #this should return max duration of all welding stations that have relationship (or name) to this vessel.
    print(f"Cycle time is {cycleTime} min")
    layRate = jointLength / (cycleTime/60/24) # should be around 3000 m/day which corresponds to cycleTime of 5.856 min for 12.2 meter long pipes
    print(f"Lay rate is {layRate} m/day")
    return layRate

# slay

In [ ]:
slay=[
    {"stationId":1, "Name":"Welding", "Description":"Root pass", "Duration":5},
    {"stationId":2, "Name":"Welding", "Description":"Hot pass", "Duration":5},
    {"stationId":3, "Name":"Welding", "Description":"Filler", "Duration":5},
    {"stationId":4, "Name":"Welding", "Description":"Cap", "Duration":5},
    {"stationId":5, "Name":"NDT", "Description":"Xray", "Duration":5},
    {"stationId":6, "Name":"FJC", "Description":"HSS", "Duration":5}
]

In [36]:
element = {
"wallThickness": 20/1000,
"od": 30*25.4/1000
}
calWelding(element, slay, weldingSpeed= 350/1000*24*60, weldingWireOD=3/1000, bevelAngle=(20*3.1459)/180, numberOfWeldMachines=2)

Welding duration is 73.55642101791453 min
Welding per station is 18.389105254478633 min
Cycle time is 18.389105254478633 min
Lay rate is 955.3482758885915 m/day


955.3482758885915

In [37]:
element = {
"wallThickness": 19/1000,
"od": 12*25.4/1000
}
calWelding(element, slay, weldingSpeed= 350/1000*24*60, weldingWireOD=3/1000, bevelAngle=(20*3.1459)/180, numberOfWeldMachines=2)

Welding duration is 26.553867987467143 min
Welding per station is 6.638466996866786 min
Cycle time is 6.638466996866786 min
Lay rate is 2646.3941160348795 m/day


2646.3941160348795

# jlay

In [41]:
jlay=[
    {"stationId":1, "Name":"Welding", "Description":"All pass", "Duration":5},
    {"stationId":2, "Name":"NDT-FJC", "Description":"Xray, Repair, HSS", "Duration":5}
]
element = {
"wallThickness": 20/1000,
"od": 30*25.4/1000
}
calWelding(element, jlay,jointLength=12.2*4, weldingSpeed= 350/1000*24*60, weldingWireOD=3/1000, bevelAngle=(20*3.1459)/180, numberOfWeldMachines=2)

Welding duration is 73.55642101791453 min
Welding per station is 73.55642101791453 min
Cycle time is 73.55642101791453 min
Lay rate is 955.3482758885915 m/day


955.3482758885915

# Reellay spool base

# spool base
Spool base has task stalk fabrication that is SF to mobilisation of the reellay vessel. This reporent the latest start date that pipes need to arrive to the spoolbase so the the reellay vessel is not delayed.

In [69]:
stalk_fabrication_spoolbase=[
    {"stationId":1, "Name":"Welding", "Description":"Root pass", "Duration":5},
    {"stationId":2, "Name":"Welding", "Description":"Hot pass", "Duration":5},
    {"stationId":3, "Name":"Welding", "Description":"Filler", "Duration":5},
    {"stationId":4, "Name":"Welding", "Description":"Cap", "Duration":5},
    {"stationId":5, "Name":"NDT", "Description":"Xray", "Duration":5},
    {"stationId":6, "Name":"FJC", "Description":"HSS", "Duration":5}
]
reel_spoolbase=[
    {"stationId":1, "Name":"Welding", "Description":"All pass", "Duration":5},
    {"stationId":2, "Name":"NDT-FJC", "Description":"Xray, Repair, HSS", "Duration":5}
]

In [71]:
element = {
"wallThickness": 20/1000,
"od": 762/1000,
"length":12.2
}
line_length = [8160] #all lines that vessel installs needs to be stalk fabricated before first vessel mobilisation
stalk_length = 1000 # the best is if this is a multiple of joints but if not we need to make it
no_of_joints_in_stalk = math.floor(stalk_length / element["length"])
stalk_length = no_of_joints_in_stalk * element["length"] # making stalk lenth to exact number of joints
line_lengths = sum(line_length)
stalk_fabrication_time = line_lengths / calWelding(element, stalk_fabrication_spoolbase,jointLength=12.2, weldingSpeed= 350/1000*24*60, weldingWireOD=3/1000, bevelAngle=(20*3.1459)/180, numberOfWeldMachines=2)
print(f"Task = Stalk fabrication, SF reellay mobilisation, Duration = {stalk_fabrication_time} days")



Welding duration is 73.55642101791453 min
Welding per station is 18.389105254478633 min
Cycle time is 18.389105254478633 min
Lay rate is 955.3482758885915 m/day
Task = Stalk fabrication, SF reellay mobilisation, Duration = 8.541387686506468 days


# reellay vessel mobilisation

In [67]:
no_of_stalk_needed = math.ceil(line_lengths / stalk_length)+1 # this is number of stalk welds, added 1 for pull head
stalk_cycle_time = stalk_length/calWelding(element, reel_spoolbase,jointLength=stalk_length, weldingSpeed= 350/1000*24*60, weldingWireOD=3/1000, bevelAngle=(20*3.1459)/180, numberOfWeldMachines=2)
stalk_welding_time = no_of_stalk_needed * stalk_cycle_time
spooling_speed = 600*24 # m/day
spooling_time = line_lengths/spooling_speed
mob_time = stalk_welding_time+spooling_time
print(f"Stalk welding = {stalk_welding_time} days")
print(f"Task = Mobilisation (spooling), Duration = {mob_time} days")

Welding duration is 73.55642101791453 min
Welding per station is 73.55642101791453 min
Cycle time is 73.55642101791453 min
Lay rate is 19345.802586743976 m/day
73.55642101791453
Stalk welding = 1.8899913733769707 days
Task = Mobilisation (spooling), Duration = 4.320546928932526 days
